In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
dados_treino = pd.read_csv("/content/drive/MyDrive/codealura/4-Word2Vec parte 1 and 2: interpretação da linguagem humana com Word embedding e treinamento de Word2Vec/treino.csv")
dados_teste = pd.read_csv("/content/drive/MyDrive/codealura/4-Word2Vec parte 1 and 2: interpretação da linguagem humana com Word embedding e treinamento de Word2Vec/teste.csv")

In [4]:
dados_treino.sample(5)

,title,text,date,category,subcategory,link
80004,Mulher de opositor venezuelano preso cobra apo...,A mulher do líder opositor venezuelano Leopold...,2015-05-30,mundo,NaN,http://www1.folha.uol.com.br/mundo/2015/05/163...
25135,Excessos dos anos 1980 marcam desfile de Lilly...,Simplificar é palavra de ordem nesta edição da...,2017-03-13,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/03...
88456,Preocupação com clima é menor em países mais p...,Os cidadãos dos países com as maiores emissões...,2016-11-01,mundo,NaN,http://www1.folha.uol.com.br/mundo/2016/01/172...
41523,Cabral diz que lei do pré-sal é responsável pe...,"O ex-governador Sérgio Cabral (PMDB), que cump...",2016-06-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/06/1...
18066,São Paulo cobra da CBF indenização de R$ 18 mi...,O São Paulo decidiu entrar na Justiça contra a...,2015-03-09,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/09/1...


In [8]:
import spacy
!python -m spacy download pt_core_news_sm # pequeno

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 42.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
nlp = spacy.load("pt_core_news_sm")
nlp

In [64]:
texto = "Rio de  3233 Janeiro %% é uma cidade maravilhosa".lower()
doc = nlp(texto)
type(doc)

spacy.tokens.doc.Doc

In [61]:
print(type(doc[0]))
print(doc[0])
print(doc.ents)
print(doc[0].is_stop)
print(doc[1].is_stop)
print(doc.ents[0].text, doc.ents[0].label_)

<class 'spacy.tokens.token.Token'>
rio
(rio de janeiro,)
False
True
rio de janeiro LOC


In [92]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino['title'])
textos_para_tratamento

<generator object <genexpr> at 0x7827d5a1fa00>

In [93]:
def trata_textos(doc):
  tokens_validos = []
  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text)
  if len(tokens_validos) > 2:
    return " ".join(tokens_validos)


In [94]:
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size=1000,
                                                        n_process=-1)]

In [95]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})

In [96]:
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [114]:
from gensim.models import Word2Vec

"""
  !!! O word2vec pode ser treinado de 2 maneiras com cbow ou skipgram !!!
  sg = 0 -> utiliza o cbow
  sg = 1 -. utiliza o skipgram
"""

w2v_modelo = Word2Vec(sg = 0, #type
                      window = 2, # quantas palavras ele considera antes e depois (para o contexto)
                      vector_size = 300, # tamanho do vetor que as palavras irão possuir
                      min_count = 5, # quantidade minimas de vezes que uma palavra deve aparecer para ser considerada
                      alpha = 0.03, # taxa de aprendizagem da rede neural
                      min_alpha = 0.007)

w2v_modelo

In [115]:
print(len(titulos_tratados))
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

84466
84466


In [122]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [124]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

INFO:gensim.models.word2vec:collecting all words and their counts
collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types


In [125]:
dir(w2v_modelo) #train

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'cbow_mean',
 'comment',
 'compute_loss',
 'corpus_count',
 

In [126]:
print(w2v_modelo.corpus_count)

84466


In [133]:
"""
Quando treinamento uma rede neural é comum acompanhar seu loss, o loss
geralmente é atualizado em cada época, mas com gensim Word2Vec não há
uma maneira direta de se fazer isso.

O método para calcular e armazenar o loss é model.get_latest_training_loss().
Porém não se calcula por época, e sim por treinamento completo. Entretanto,
podemos driblar este problema configurando uma mensagem de callback.
"""
# w2v_modelo.train(lista_lista_tokens,
#                 total_examples=w2v_modelo.corpus_count,
#                 epochs = 30,
#                 compute_loss = True,
#                 callbacks=[callback()])

'\nQuando treinamento uma rede neural é comum acompanhar seu loss, o loss \ngeralmente é atualizado em cada época, mas com gensim Word2Vec não há \numa maneira direta de se fazer isso.\n\nO método para calcular e armazenar o loss é model.get_latest_training_loss(). \nPorém não se calcula por época, e sim por treinamento completo. Entretanto, \npodemos driblar este problema configurando uma mensagem de callback.\n'

In [127]:
#cbow
w2v_modelo.train(lista_lista_tokens,
                 total_examples=w2v_modelo.corpus_count,
                 epochs=30)

INFO:gensim.models.word2vec:EPOCH 0 - PROGRESS: at 55.60% examples, 264231 words/s, in_qsize 5, out_qsize 0
EPOCH 0 - PROGRESS: at 55.60% examples, 264231 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 0: training on 540242 raw words (486138 effective words) took 1.7s, 293406 effective words/s
EPOCH 0: training on 540242 raw words (486138 effective words) took 1.7s, 293406 effective words/s
INFO:gensim.models.word2vec:EPOCH 1 - PROGRESS: at 64.83% examples, 310050 words/s, in_qsize 5, out_qsize 0
EPOCH 1 - PROGRESS: at 64.83% examples, 310050 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 1: training on 540242 raw words (486198 effective words) took 1.5s, 328005 effective words/s
EPOCH 1: training on 540242 raw words (486198 effective words) took 1.5s, 328005 effective words/s
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 64.83% examples, 313233 words/s, in_qsize 5, out_qsize 0
EPOCH 2 - PROGRESS: at 64.83% examples, 313233 words/s, in_qsize

(14584482, 16207260)

In [128]:
w2v_modelo.wv.most_similar("google")

[('apple', 0.5807647705078125),
 ('facebook', 0.546555757522583),
 ('amazon', 0.49353182315826416),
 ('airbnb', 0.482377827167511),
 ('uber', 0.46897026896476746),
 ('yahoo', 0.45929211378097534),
 ('fbi', 0.4578205347061157),
 ('volkswagen', 0.45138341188430786),
 ('tesla', 0.4481939375400543),
 ('waze', 0.4459397494792938)]

In [129]:
w2v_modelo.wv.most_similar("microsoft")

[('unilever', 0.5926939249038696),
 ('amazon', 0.5507664084434509),
 ('tesla', 0.5371326208114624),
 ('buffett', 0.5351148843765259),
 ('sony', 0.5322896838188171),
 ('walmart', 0.5321263074874878),
 ('linkedin', 0.5262803435325623),
 ('moma', 0.5059595704078674),
 ('braskem', 0.505673348903656),
 ('sky', 0.5036970973014832)]

In [130]:
w2v_modelo.wv.most_similar("barcelona")

[('bayern', 0.5649645924568176),
 ('barça', 0.551959216594696),
 ('chelsea', 0.5462777018547058),
 ('munique', 0.5438597202301025),
 ('lazio', 0.5350853204727173),
 ('madrid', 0.5301969647407532),
 ('psg', 0.5239397287368774),
 ('united', 0.5236823558807373),
 ('juventus', 0.5195978879928589),
 ('liverpool', 0.5131831765174866)]

In [131]:
w2v_modelo.wv.most_similar("messi")

[('suárez', 0.5915347337722778),
 ('chuteiras', 0.5317542552947998),
 ('cavani', 0.5307847857475281),
 ('cristiano', 0.5253778696060181),
 ('tevez', 0.5243858695030212),
 ('neymar', 0.5179855227470398),
 ('barça', 0.49488475918769836),
 ('ronaldo', 0.4928181767463684),
 ('benzema', 0.4908091127872467),
 ('barcelona', 0.4876478612422943)]

In [132]:
w2v_modelo.wv.most_similar("gm")

[('chrysler', 0.6697890758514404),
 ('embraer', 0.6513582468032837),
 ('volks', 0.6440757513046265),
 ('honda', 0.6437822580337524),
 ('braskem', 0.6411211490631104),
 ('fiat', 0.6120463609695435),
 ('tesla', 0.5956095457077026),
 ('csn', 0.5952708125114441),
 ('toyota', 0.58732670545578),
 ('volkswagen', 0.5783541202545166)]

In [134]:
#skipgram
w2v_modelo_sg = Word2Vec(sg = 1,
                      window = 5,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

#skipgram - train
w2v_modelo_sg.train(lista_lista_tokens,
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs=30)

INFO:gensim.models.word2vec:collecting all words and their counts
collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types


(14584636, 16207260)

In [140]:
print(w2v_modelo_sg.wv.most_similar("google"))
print(w2v_modelo.wv.most_similar("google"), "\n")
print(w2v_modelo_sg.wv.most_similar("gm"))
print(w2v_modelo.wv.most_similar("gm"), "\n")

[('reguladores', 0.4141896367073059), ('android', 0.40323880314826965), ('waze', 0.37738075852394104), ('buffett', 0.3767269551753998), ('anúncios', 0.36980196833610535), ('toshiba', 0.3681389093399048), ('apple', 0.3590114712715149), ('concorda', 0.3561571538448334), ('patentes', 0.3524250388145447), ('yahoo', 0.35239115357398987)]
[('apple', 0.5807647705078125), ('facebook', 0.546555757522583), ('amazon', 0.49353182315826416), ('airbnb', 0.482377827167511), ('uber', 0.46897026896476746), ('yahoo', 0.45929211378097534), ('fbi', 0.4578205347061157), ('volkswagen', 0.45138341188430786), ('tesla', 0.4481939375400543), ('waze', 0.4459397494792938)] 

[('metalúrgicos', 0.5725686550140381), ('motors', 0.5062864422798157), ('honda', 0.49331676959991455), ('airbags', 0.48092034459114075), ('bmw', 0.47661298513412476), ('mitsubishi', 0.47452470660209656), ('audi', 0.47215402126312256), ('chrysler', 0.47092926502227783), ('coletivas', 0.4696614742279053), ('airbag', 0.4675992429256439)]
[('chry

In [141]:
w2v_modelo.wv.save_word2vec_format("/content/drive/MyDrive/codealura/4-Word2Vec parte 1 and 2: interpretação da linguagem humana com Word embedding e treinamento de Word2Vec/modelo_cbow.txt", binary=False)

w2v_modelo_sg.wv.save_word2vec_format("/content/drive/MyDrive/codealura/4-Word2Vec parte 1 and 2: interpretação da linguagem humana com Word embedding e treinamento de Word2Vec/modelo_skipgram.txt", binary=False)
